In [104]:
# Homework 4 (due 07/24/2024)

# Decision trees, interpretability, and algorithmic bias

## Objective

In this week's project, you will explore the COMPAS data set. COMPAS stands for "Correctional Offender Management Profiling for Alternative Sanctions". It is a software/algorithm that is used to assess the risk of a registered offender is going to commit another offense. Although researchers and journalists have pointed to [various problems](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) of this algorithm over many years, the algorithm is still used to inform sentences and parole decisions in several US states. 
You can learn more about the COMPAS data set [here](https://www.propublica.org/datastore/dataset/compas-recidivism-risk-score-data-and-analysis). 

Through this project, you will practice fitting and validating several classification models and you will explore some distinct benefits of using decision trees in machine learning. As part of that exploration, you are going to audit your model for demographic biases via a "closed box" and an "open box" approach.

The COMPAS data set is a favorite example among critics of machine learning because it demonstrates several shortcomings and failure modes of machine learning techniques. The lessons learned from this project might be discouraging, and they are important. Keep in mind, however, that what you see here does not generalize to all data sets. 

This project has four parts.

### Part 1: Prepare the COMPAS data set  (PARTIALLY YOU TO COMPLETE)

In this part, you will load the COMPAS data set, explore its content, and select several variables as features (i.e., queries) or class labels (i.e., responses). Some of these features are not numerical, so you will need to replace some categorical values with zeros and ones. Your features will include categorical variable with more than two categories. You will uses 1-hot encoding to include this feature in your data set. 

This part includes four steps:
1. Load and explore data set
2. Select features and response variables
3. Construct numerical coding for categorical features
4. Split the data

### Part 2: Train and validate a decision tree  (PARTIALLY YOU TO COMPLETE)

In this part, you will fit a decision tree to your data. You will examine the effect of tuning the complexity of the tree via the "maximum number of leaves" parameter and use 5-fold cross-validation to find an optimal value.

This part includes three steps:

1. Fit a decision tree on the training data
2. Tune the parameter "maximum number of leaves"
3. Calculate the selected model's test performance


### Part 3: Auditing a decision tree for demographic biases  (PARTIALLY YOU TO COMPLETE)

Your training data includes several demographic variables (i.e., age, sex, race). A crude way to assess whether a model has some demographic bias is to remove the corresponding variables from your training data and explore how that removal affects your model's performance. Decision trees have the advantage of being interpretable machine learning models. By going through the decision nodes (i.e., branching points), you can "open the black box and look inside". Specifically, you can assess how each feature is used in the decision making process.

This part includes three steps:

1. Fit a decision tree
2. Check for racial bias via performance assessment
3. Check for racial bias via decision rules

### Part 4: Comparison to other linear classifiers (FOR YOU TO COMPLETE)

For some types of data, decision trees tend to achieve lower prediction accuracies In this part, you will train and tune several classifiers on the COMPAS data. You will then compare their performance on your test set.

This part includes three steps:

1. Fit LDA and logistic regression
2. Tune and fit ensemble methods
3. Tune and fit SVC
4. Compare performance metrics for all models 

In [105]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

## Part 1: Prepare the COMPAS data set

>In this part, you will load the COMPAS data set, explore its content, and select several variables as features (i.e., queries) or class labels (i.e., responses). Some of these features are not numerical, so you will need to replace some categorical values with zeros and ones. Your features will include categorical variable with more than two categories. You will uses 1-hot encoding to include this feature in your data set.
>
>This part includes four steps:
>1. Load and explore data set
>2. Select features and response variables
>3. Construct numerical coding for categorical features
>4. Split the data



### Part 1, Step 1: Load and explore data set

This folder includes the 'compas-scores-two-years.csv' file. The COMPAS data that you will use for this project is in this file. It is always a good idea to look at the raw data before proceeding with one's machine learning pipeline.

In [106]:
# load data
raw_data = pd.read_csv('compas-scores-two-years.csv')
# print a list of variable names
print(raw_data.columns)
# look at the first 5 rows 
raw_data.head(5)

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


The data set includes 53 variables. There are different types of information. Some variables
* personal data (e.g., name, first name ("first"), last name ("last")) 
* demographic data (i.e., sex, age, age category ("age_cat"), and race)
* related to the person's history of commited offenses (e.g., juvenile felony count ("juv_fel_count"), juvenile misdemeanor count ("juv_misd_count"), and prior offenses count ("priors-count"))
* related to the charge against the person (e.g., charge offense date ("c_offense_date"), charge arrest date ("c_arrest_date"), charge degree ("c_charge_degree"), and description of charge ("c_charge_desc"))
* recidivism scores assigned by the COMPAS algorithm (e.g., "decile_score", "score_text", "v_decile_score", "v_score_text")
* related to an actual recidivism charge (e.g., degree of recidivism charge ("r_charge_degree"), data of recidivism offense ("r_offense_date"), description of recidivism charge ("r_charge_desc"))
* related to an actual violent recidivism charge (e.g., degree of violent recidivism charge ("vr_charge_degree"), data of violent recidivism offense ("vr_offense_date"), description of violent recidivism charge ("vr_charge_desc")).

### Part 1, Step 2: Select features and response variables

The ProPublica article was assessing bias in the COMPAS scores. Here, you will ignore the COMPAS scores and instead explore the challenges of predicting recidivism based on the survey data. What variables seem like sensible predictors? What variables would be sensible outcome variables? The code in the cell below selects some numerical and categorical variables for you to include in your model.

In [107]:
# Select features and response variables

# Features by type
numerical_features = ['juv_misd_count', 'juv_other_count', 'juv_fel_count', 
    'priors_count', 'age']
binary_categorical_features = ['sex', 'c_charge_degree']
other_categorical_features = ['race']
all_features = binary_categorical_features + other_categorical_features + numerical_features

# Possible esponse variables
response_variables = ['is_recid', 'is_violent_recid', 'two_year_recid']

# Variables that are used for data cleaning
check_variables = ['days_b_screening_arrest']

ProPublica filtered some observations (i.e., rows in the data frame). See their explanation below. Let's follow their procedure.


> There are a number of reasons remove rows because of missing data:
>
> * If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
> * We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
> * In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
> * We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.


In [108]:
# Subselect data
df = raw_data[all_features+response_variables+check_variables]

# Apply filters
df = df[(df['days_b_screening_arrest'] <= 30) & 
        (df['days_b_screening_arrest'] >= -30) & 
        (df['is_recid'] != -1) & 
        (df['c_charge_degree'] != 'O')]

df = df[all_features+response_variables]
print('Dataframe has {} rows and {} columns.'.format(df.shape[0], df.shape[1]))

Dataframe has 6172 rows and 11 columns.


### Part 1, Step 3: Construct numerical coding for categorical features

Some of these features in the subselected data are not numerical, so you will need to replace some categorical values with zeros and ones. Your features will include "race", which was surveyed as a one categorical variable with more than two categories. You will uses [1-hot encoding](https://en.wikipedia.org/wiki/One-hot) to include this feature in your data set. 

In [109]:
# Code binary features as 0 and 1
for x in binary_categorical_features:
    for new_value, value in enumerate(set(df[x])):
        print("Replace {} with {}.".format(value, new_value))
        df = df.replace(value, new_value)

Replace Female with 0.
Replace Male with 1.
Replace F with 0.
Replace M with 1.


/var/folders/kl/1_fncc2d60bch9r3xcs1m_940000gn/T/ipykernel_47281/1287833026.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(value, new_value)


In [110]:
# Use 1-hot encoding for other categorical variables
one_hot_features = []
for x in other_categorical_features:
    for new_feature, value in enumerate(set(df[x])):
        feature_name = "{}_is_{}".format(x,value)
        df.insert(3, feature_name, df[x]==value)
        one_hot_features += [feature_name]

# Check what the data frame looks like now
df.head(10)

,sex,c_charge_degree,race,race_is_Hispanic,race_is_Caucasian,race_is_Other,race_is_Native American,race_is_African-American,race_is_Asian,juv_misd_count,juv_other_count,juv_fel_count,priors_count,age,is_recid,is_violent_recid,two_year_recid
0,1,0,Other,False,False,True,False,False,False,0,0,0,0,69,0,0,0
1,1,0,African-American,False,False,False,False,True,False,0,0,0,0,34,1,1,1
2,1,0,African-American,False,False,False,False,True,False,0,1,0,4,24,1,0,1
5,1,1,Other,False,False,True,False,False,False,0,0,0,0,44,0,0,0
6,1,0,Caucasian,False,True,False,False,False,False,0,0,0,14,41,1,0,1
7,1,0,Other,False,False,True,False,False,False,0,0,0,3,43,0,0,0
8,0,1,Caucasian,False,True,False,False,False,False,0,0,0,0,39,0,0,0
10,1,0,Caucasian,False,True,False,False,False,False,0,0,0,0,27,0,0,0
11,1,1,African-American,False,False,False,False,True,False,0,0,0,3,23,1,0,1
12,0,1,Caucasian,False,True,False,False,False,False,0,0,0,0,37,0,0,0


### Part 1, Step 4: Split the data

Let's collect the features in one data frame and the responses in another data frame. After that, you will set a small portion of the data set aside for testing.

In [115]:
# list of features
features = numerical_features + binary_categorical_features + one_hot_features

# features data frame
X = df[features]

# responses data frame
Y = df[response_variables]

# Split the data into a training set containing 90% of the data
# and test set containing 10% of the data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

# Part 2: Train and validate a decision tree

>In this part, you will fit a decision tree to your data. You will examine the effect of tuning the complexity of the tree via the "maximum number of leaves" parameter and use 5-fold cross-validation to find an optimal value.
>
>This part includes three steps:
>
>1. Fit a decision tree on the training data
>2. Tune the parameter "maximum number of leaves"
>3. Calculate the selected model's test performance

### Part 2, Step 1: Fit a decision tree on the training data

Start by fitting a decision tree to your training data. Assess its training accuracy and its size.

In [116]:
# Create a model
dtc = DecisionTreeClassifier()

# Fit model to training data
dtc = dtc.fit(X_train, Y_train['two_year_recid'])    

# Evaluate training accuracy
prediction_values = dtc.predict(X_train)

# average by counting "1"s (accurate results) and dividing by total number of observations
count = 0
for i in range(prediction_values.size):
    if prediction_values[i] == Y_train['two_year_recid'].iloc[i]:
        count = count + 1

accuracy = count / prediction_values.size

# Check size of decision tree
num_leaves = dtc.get_n_leaves()

# Report results
print('Trained decision tree with {} leaves and training accuracy {:.2f}.'.format(num_leaves, accuracy))

Trained decision tree with 1773 leaves and training accuracy 0.84.


Your tree has a good training accuracy for the standards of tabular data prediction problems, but its size is enormous! It has so many leaves, that on average every 3 to 4 training observations get a leaf to themselves. It is very probable that this tree is overfitting.

### Part 2, Step 2: Tune the parameter "maximum number of leaves"

Let's try to constrain the complexity of a decision tree during training by setting a value for the argument ``maximum number of leaves``. You can use the sci-kit learn's `cross_val_score` function to quickly assess the out-of-sample performance of trees of varying complexity.

In [117]:
# Perform 5-fold cross-validation for different tree sizes

print('Leaves\tMean accuracy')
print('---------------------')

# initiate best max and accuracy as 0 and 0
best_max_leaves = 0
best_accuracy = 0

# try all leaves from 0-1800
for num_leaves in range(0,1800,1):

        # Trees must have at least 2 leaves
        if num_leaves >= 2:

            # construct a classifier with a limit on its number of leaves
            dtc = DecisionTreeClassifier(max_leaf_nodes=num_leaves, random_state=5).fit(X_train, Y_train['two_year_recid'])
            
            # Get validation accuracy via 5-fold cross-validation
            scores = cross_val_score(dtc, X_train, Y_train['two_year_recid'])
        
        print("{}\t{:.3f}".format(num_leaves,scores.mean()))

        # compare accuracy, update best max leaves and best accuracy if better than current
        if scores.mean() > best_accuracy:
              best_max_leaves = num_leaves
              best_accuracy = scores.mean()

print("best max for leaves is", best_max_leaves, " with an accuracy of ", best_accuracy)

Leaves	Mean accuracy
---------------------
0	0.655
1	0.655
2	0.655
3	0.654
4	0.654
5	0.658
6	0.668
7	0.673
8	0.673
9	0.673
10	0.675
11	0.676
12	0.676
13	0.677
14	0.677
15	0.681
16	0.681
17	0.685
18	0.684
19	0.684
20	0.685
21	0.685
22	0.685
23	0.683
24	0.683
25	0.683
26	0.684
27	0.683
28	0.683
29	0.683
30	0.683
31	0.683
32	0.683
33	0.684
34	0.684
35	0.684
36	0.684
37	0.684
38	0.685
39	0.685
40	0.685
41	0.685
42	0.685
43	0.685
44	0.685
45	0.685
46	0.685
47	0.685
48	0.685
49	0.682
50	0.682
51	0.682
52	0.683
53	0.683
54	0.682
55	0.682
56	0.682
57	0.682
58	0.682
59	0.680
60	0.681
61	0.681
62	0.680
63	0.680
64	0.680
65	0.679
66	0.679
67	0.678
68	0.677
69	0.676
70	0.675
71	0.675
72	0.675
73	0.675
74	0.675
75	0.675
76	0.675
77	0.674
78	0.674
79	0.674
80	0.675
81	0.674
82	0.674
83	0.674
84	0.674
85	0.674
86	0.674
87	0.674
88	0.674
89	0.674
90	0.674
91	0.674
92	0.673
93	0.673
94	0.674
95	0.674
96	0.674
97	0.674
98	0.672
99	0.672
100	0.672
101	0.672
102	0.671
103	0.671
104	0.670
105	0.670
106	0.6

Adjust the range of values for `max_leaf_nodes` in the cell above, to identify the best value.

### Part 2, Step 3: Calculate the selected model's test performance

Train a decision tree using your selected value of `max_leaf_nodes` on the full training set. Assess its accuracy on your test set.

In [141]:
max_leaves = best_max_leaves

# Create a model w/ best max leaves
dtc = DecisionTreeClassifier(max_leaf_nodes=max_leaves)

# Fit model to training data
dtc = dtc.fit(X_train, Y_train['two_year_recid'])

# Evaluate training accuracy
prediction_values = dtc.predict(X_test)

# average by counting "1"s (accurate results) and dividing by total number of observations
count = 0
for i in range(prediction_values.size):
    if prediction_values[i] == Y_test['two_year_recid'].iloc[i]:
        count = count + 1

dtc_accuracy = count / prediction_values.size

# Check size of decision tree
num_leaves = dtc.get_n_leaves()

print(num_leaves)

# Report results
print('Trained decision tree with {} leaves and test accuracy {:.2f}.'.format(num_leaves, dtc_accuracy))

41
Trained decision tree with 41 leaves and test accuracy 0.66.


# Part 3: Auditing a decision tree for demographic biases

>Your training data includes several demographic variables (i.e., age, sex, race). A crude way to assess whether a model has some demographic bias is to remove the corresponding variables from your training data and explore how that removal affects your model's performance. Decision trees have the advantage of being interpretable machine learning models. By going through the decision nodes (i.e., branching points), you can "open the black box and look inside". Specifically, you can assess how each feature is used in the decision making process.
>
>This part includes two steps:
>
>1. Check for racial bias via performance assessment
>2. Check for racial bias via decision rules
  
### Part 3, Step 2: Check for racial bias via performance assessment
A simple approach to identifying demographic biases in machine learning is the following: (i) Train and validate the model on the full training set, (ii) train and validate the model on a subset of training variables that excludes the variables related to a potential demographic bias, (iii) compare the results. 

You have noticed that the validation accuracy of your model can vary for different holdout set selections. To account for these variations, you are going to compare the mean validation accuracy over 100 trees. (You have completed (i) in the previous cell already. Continue now with (ii).)

In [142]:
# Create subset of training data without information on race. 
# (The information on race was encoded in the one-hot features.)
remaining_features = [v for v in X.columns if v not in one_hot_features]
X_train_sub = X_train[remaining_features]
X_test_sub = X_test[remaining_features]

# Create a model
dtc_sub = DecisionTreeClassifier(max_leaf_nodes=best_max_leaves)
    
# Fit model to training data
dtc_sub.fit(X_train_sub, Y_train['two_year_recid'])

# Evaluate training accuracy
y_pred = dtc_sub.predict(X_test_sub)
accuracy = (y_pred == Y_test['two_year_recid']).mean()

# cross-validate over 100 times
scores = cross_val_score(dtc_sub, X_train_sub, Y_train['two_year_recid'])

# compute accuracy for scores from each iteration
accuracy = scores.mean()

# Check size of decision tree
num_leaves = dtc_sub.get_n_leaves()

# Report results
print('Trained decision tree with {} leaves and test accuracy {:.2f}.'.format(num_leaves, accuracy))

# The mean accuracy for the subselected feature set (~0.68) is slightly better than the mean accuracy for all-features set (~0.66)

Trained decision tree with 41 leaves and test accuracy 0.68.


Comparing the mean accuracy values on the all features versus the subselected feature set, what do you conclude about the importance of racial information in this classification problem?

### Part 2, Step 3: Check for racial bias via decision rules
The interpretability of decision trees allows for an alternative approach to detecting racial bias. You can simply look at the decision rules. Use the scit-kit learn's function `export_text` to get your decision tree in text format. Compare the decision rules of the your tree with all features and your tree fitted on the subset without racial information. Do you find any indication of racial bias in the decision rules of the first tree?

In [143]:
print(export_text(dtc, feature_names = X_train.columns))
print("------------------------------------------------------------------------------------------------")
print(export_text(dtc_sub, feature_names = X_train_sub.columns))
# One obvious difference when comparing the two is having decision rules based on race in the first tree which are absent in the second.
# Specifically, when examining the decision rules of the first tree, I found a several instances of racial bias.
# If a subject is between 22.5 and 26.5, has a priors count between 0.5 and 1.5, and is male, then being African-American determines whether the subject
# is predicted to recidivism within 2 years: if African-American, the decision tree predicts yes, if not, the decision tree predicts no.
# Another example: If a subject is between 22.5 and 24.5, has a priors count between 1.5 and 2.5, then being African-American determines whether the subject
# is predicted to recidivism within 2 years: if African-American, the decision tree predicts yes, if not, the decision tree predicts no.
# I found four examples of decision rules making different outcomes based on race (3 times if African-American, 1 time based on whether the subject was white)

|--- priors_count <= 2.50
|   |--- age <= 28.50
|   |   |--- age <= 22.50
|   |   |   |--- age <= 20.50
|   |   |   |   |--- priors_count <= 0.50
|   |   |   |   |   |--- juv_other_count <= 1.50
|   |   |   |   |   |   |--- age <= 19.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- age >  19.50
|   |   |   |   |   |   |   |--- sex <= 0.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- sex >  0.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- juv_other_count >  1.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- priors_count >  0.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- age >  20.50
|   |   |   |   |--- sex <= 0.50
|   |   |   |   |   |--- age <= 21.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- age >  21.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- sex >  0.50
|   |   |   |   |   |--- priors_count <= 1.50
|   |   |   |   |   |   |--- class: 1
| 

# Part 4: Comparison to other linear classifiers

>For some types of data, decision trees tend to achieve lower prediction accuracies In this part, you will train and tune several classifiers on the COMPAS data. You will then compare their performance on your test set.
>
>This part includes three steps:
>
>1. Fit LDA and logistic regression
>2. Tune and fit ensemble methods
>3. Tune and fit SVC
>4. Compare test accuracy of all your models 

In [147]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

# make dictionary to compare all methods
all_accuracies = {}
# add our decision tree
all_accuracies["DTC"] = dtc_accuracy





# 1. LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, Y_train['two_year_recid'])

# Evaluate training accuracy and print
y_pred = lda.predict(X_test)
accuracy = (y_pred == Y_test['two_year_recid']).mean()
print("LDA accuracy: ", accuracy)
# add to dictionary of all test methods
all_accuracies["LDA"] = accuracy


# fit logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train['two_year_recid'])

# Evaluate training accuracy and print
y_pred = logreg.predict(X_test)
accuracy = (y_pred == Y_test['two_year_recid']).mean()
print("LogReg accuracy: ", accuracy)
all_accuracies["LogReg"] = accuracy





# 2. Tune and fit ensemble methods
# Random Forest
best_accuracy = 0
best_n = 0
estimator_range = 200
increment = 10
start = 10
# tune based on number of estimators (n_estimators)
for i in range(start,estimator_range,increment):
    rand_for = RandomForestClassifier(n_estimators=i, random_state=5)
    rand_for.fit(X_train, Y_train['two_year_recid'])

    # Evaluate training accuracy and print
    y_pred = rand_for.predict(X_test)
    accuracy = (y_pred == Y_test['two_year_recid']).mean()

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_n = i

print("RandomForest accuracy: ", best_accuracy, "with best n ", best_n)
all_accuracies["RandomForest"] = best_accuracy





# Bagging
best_accuracy = 0
best_n = 0
estimator_range = 200
increment = 10
start = 10
# tune based on number of estimators (n_estimators)
for i in range(start,estimator_range,increment):
    bag = BaggingClassifier(n_estimators=i, random_state=5)
    bag.fit(X_train, Y_train['two_year_recid'])

    # Evaluate training accuracy and print
    y_pred = bag.predict(X_test)
    accuracy = (y_pred == Y_test['two_year_recid']).mean()

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_n = i

print("Bagging accuracy: ", best_accuracy, "with best n", best_n)
all_accuracies["Bagging"] = best_accuracy






# GradientBoosting
best_accuracy = 0
best_lr = 0
lr_range = 100
# tune based on learning rate (0.01-1 in increments of 0.01)
for i in range(lr_range):
    lr = 0.01 * i
    GradBoost = GradientBoostingClassifier(learning_rate=lr, random_state=5)
    GradBoost.fit(X_train, Y_train['two_year_recid'])

    # Evaluate training accuracy and print
    y_pred = GradBoost.predict(X_test)
    accuracy = (y_pred == Y_test['two_year_recid']).mean()

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lr = i*0.01
print("GradientBoosting accuracy: ", best_accuracy, " with learning rate ", best_lr)
all_accuracies["GradientBoosting"] = best_accuracy





# 3. Tune + fit SVC
# tune for C = 0.01, 0.1, 1, 10, 100
best_accuracy = 0
best_C = 0
powers = 5
for i in range(powers):
    # adjust C value (increase by power of 10 each time)
    C_value = 1 * 10**(-2 + i)
    svc = SVC(C=C_value)
    svc.fit(X_train, Y_train['two_year_recid'])

    # Evaluate training accuracy and print
    y_pred = svc.predict(X_test)
    accuracy = (y_pred == Y_test['two_year_recid']).mean()

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C_value

print("SVC accuracy (best C): ", best_accuracy)
all_accuracies["SVC"] = best_accuracy


overall_best_accuracy = 0
best_method = ""
for i in all_accuracies:
    if all_accuracies[i] > overall_best_accuracy:
        overall_best_accuracy = all_accuracies[i]
        best_method = i

print("Best method is:", best_method, ", with accuracy:", overall_best_accuracy)

LDA accuracy:  0.63915857605178


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogReg accuracy:  0.6423948220064725
RandomForest accuracy:  0.627831715210356 with best n  50
Bagging accuracy:  0.6310679611650486 with best n 50
GradientBoosting accuracy:  0.6715210355987055  with learning rate  0.02
SVC accuracy (best C):  0.6618122977346278
Best method is: GradientBoosting , with accuracy: 0.6715210355987055
